In [244]:
import requests
import pandas as pd
import plotly.express as px
import plotly
colors = px.colors.qualitative.D3 + px.colors.qualitative.Dark24 + plotly.colors.DEFAULT_PLOTLY_COLORS + px.colors.qualitative.Plotly + px.colors.qualitative.Alphabet

In [183]:
def download_data():
    url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
    metadata = requests.get(url_metadata)
    
    with open('data/france/metadata.csv', 'wb') as f:
        f.write(metadata.content)
        
    df_metadata = pd.read_csv('data/france/metadata.csv', sep=";")
    url_data = df_metadata[df_metadata['url'].str.contains("/donnees-hospitalieres-covid19")]["url"].values[0]
    
    data = requests.get(url_data)
    with open('data/france/donnes-hospitalieres-covid19.csv', 'wb') as f:
        f.write(data.content)

download_data()

In [289]:
def import_data():
    df = pd.read_csv('data/france/donnes-hospitalieres-covid19.csv', sep=";")
    df_regions = pd.read_csv('data/france/departments_regions_france_2016.csv', sep=",")
    df_pop = pd.read_csv('data/france/population_regions.csv', sep=",")
    df = df.merge(df_regions, left_on='dep', right_on='departmentCode')
    df = df.merge(df_pop, left_on='regionName', right_on='regionName')
    
    df['rea_pop'] = df['rea']/df['regionPopulation']*100000
    df['rad_pop'] = df['rad']/df['regionPopulation']*100000
    df['dc_pop'] = df['dc']/df['regionPopulation']*100000
    df['hosp_pop'] = df['hosp']/df['regionPopulation']*100000
    return df
df = import_data()
df = df[df["sexe"] == 0]
df_region = df.groupby(['regionName', 'jour']).sum().reset_index()
#df_region["dc_cumsum"] = df.groupby(by=['regionName','jour']).sum().groupby(level=[0]).cumsum().reset_index()["dc"]

df

,dep,sexe,jour,hosp,rea,rad,dc,departmentCode,departmentName,regionCode,regionName,regionPopulation,rea_pop,rad_pop,dc_pop,hosp_pop
0,01,0,2020-03-18,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,0.000000,0.024209,0.0,0.048418
3,01,0,2020-03-19,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,0.000000,0.024209,0.0,0.048418
6,01,0,2020-03-20,2,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,0.000000,0.024209,0.0,0.048418
9,01,0,2020-03-21,4,0,1,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,0.000000,0.024209,0.0,0.096835
12,01,0,2020-03-22,8,1,2,0,01,Ain,84.0,Auvergne-Rhône-Alpes,4130719,0.024209,0.048418,0.0,0.193671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409,976,0,2020-03-21,0,0,0,0,976,Mayotte,6.0,Mayotte,146262,0.000000,0.000000,0.0,0.000000
2412,976,0,2020-03-22,0,0,0,0,976,Mayotte,6.0,Mayotte,146262,0.000000,0.000000,0.0,0.000000
2415,976,0,2020-03-23,0,0,0,0,976,Mayotte,6.0,Mayotte,146262,0.000000,0.000000,0.0,0.000000
2418,976,0,2020-03-24,0,0,0,0,976,Mayotte,6.0,Mayotte,146262,0.000000,0.000000,0.0,0.000000


In [350]:
fig = px.line(x=df_region['jour'], y=df_region['rea']+df_region['hosp'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.)'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés")
fig.show()

In [349]:
fig = px.line(x=df_region['jour'], y=df_region['rea_pop']+df_region['hosp_pop'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.) pour 100k habitants'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés / 100k habitants")
fig.show()

In [348]:
df_rhin = df[df["dep"].isin(["67", "68"])]
fig = px.line(x=df_rhin['jour'], y=df_rhin['rea'], color=df_rhin["dep"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients en réa. ou soins intensifs par dép.'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.show()

In [347]:

fig = px.line(x=df['jour'], y=df['rea'], color=df["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients en réa. ou soins intensifs par dép.'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients en réa. ou soins intensifs")
fig.show()

In [346]:

fig = px.line(x=df_region['jour'], y=df_region['rea_pop']+df_region['hosp_pop'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.) pour 100k habitants de chaque région'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients hospitalisés / 100k habitants (de ch. région)")
fig.show()

In [343]:
fig = px.line(x=df_region['jour'], y=df_region['dc'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès cumulés'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.show()

In [342]:
fig = px.line(x=df_region['jour'], y = df_region['dc']/df_region['regionPopulation']*100000, color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès cumulés pour 100k habitants de chaque région'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de décès cumulés pour 100k habitants (de chaque région)")
fig.show()

In [341]:
fig = px.bar(x=df_region['jour'], y = df_region['dc'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès journaliers'
)
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de décès journaliers")
fig.show()
df_region[df_region["regionName"] == "Auvergne-Rhône-Alpes"]

,regionName,jour,sexe,hosp,rea,rad,dc,regionCode,regionPopulation,rea_pop,rad_pop,dc_pop,hosp_pop
0,Auvergne-Rhône-Alpes,2020-03-18,0,172,35,69,7,1008.0,49568628,0.847310,1.670411,0.169462,4.163924
1,Auvergne-Rhône-Alpes,2020-03-19,0,385,79,132,22,1008.0,49568628,1.912500,3.195570,0.532595,9.320411
2,Auvergne-Rhône-Alpes,2020-03-20,0,463,87,179,33,1008.0,49568628,2.106171,4.333386,0.798892,11.208702
3,Auvergne-Rhône-Alpes,2020-03-21,0,525,88,192,38,1008.0,49568628,2.130380,4.648101,0.919937,12.709652
4,Auvergne-Rhône-Alpes,2020-03-22,0,641,106,212,47,1008.0,49568628,2.566139,5.132278,1.137816,15.517880
5,Auvergne-Rhône-Alpes,2020-03-23,0,853,157,254,70,1008.0,49568628,3.800791,6.149051,1.694620,20.650158
6,Auvergne-Rhône-Alpes,2020-03-24,0,981,190,314,86,1008.0,49568628,4.599683,7.601582,2.081962,23.748892
7,Auvergne-Rhône-Alpes,2020-03-25,0,1230,258,404,107,1008.0,49568628,6.245886,9.780380,2.590348,29.776898


In [340]:
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title_text='Nombre de décès cumulés'
)
fig.update_xaxes(title="Région")
fig.update_yaxes(title="Nb. de décès cumulés")
fig.show()

In [351]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea', 'hosp'])
df_bar = df_bar.groupby(['jour', 'variable']).sum().reset_index()
fig = px.bar(df_bar, x='jour', y='value', color="variable")
fig.update_layout(
    title_text='Nombre de personnes hospitalisées (dont réanimation ou soins intensifs)'
)
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="Jour")
fig.update_yaxes(title="Nb. de patients actuellement hospitalisés")
fig.show()

In [335]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea_pop', 'hosp_pop'])
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable")
#fig.show()
df_bar

xs = df_bar['regionName'].values
ys = df_bar['value'].values
color = df_bar['variable'].values


In [363]:
last_day = df['jour'].max()
df_bar = pd.melt(df, id_vars=['jour','regionName'], value_vars=['rea_pop', 'hosp_pop'])
df_bar = df_bar[df_bar['jour']==last_day]
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable").update_xaxes(categoryorder="total descending")

fig.update_layout(
    title_text='Nombre de patients hospitalisés et en réanimation pour 100k hab le 25 mars',
    title={
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
),
titlefont = dict(
            size=40),
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'
fig.update_xaxes(title="Région")
fig.update_yaxes(title="Nb. de patients actuellement hospitalisés")
fig.show()